# Predicting Porosity

The example explains how to estimate porosity of a network. We also discuss some challenges in estimating the porosity of the network and how to reduce the estimation error.

In [1]:
import numpy as np
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
np.random.seed(10)
%matplotlib inline
np.set_printoptions(precision=5)

### Create a random cubic network

In [2]:
pn = op.network.Cubic(shape=[15, 15, 15], spacing=1e-6)
pn.add_model_collection(op.models.collections.geometry.cubes_and_cuboids)
pn.regenerate_models()

### Calculate total void volume

In [3]:
Vol_void = np.sum(pn['pore.volume'])+np.sum(pn['throat.volume'])

### Calculate total domain volume

In [4]:
inlet = pn.pores('left')
outlet = pn.pores('right')
A = op.topotools.get_domain_area(pn, inlets=inlet, outlets=outlet)
L = op.topotools.get_domain_length(pn, inlets=inlet, outlets=outlet)
Vol_bulk = A * L
Poro = Vol_void / Vol_bulk
print(f'The value of Porosity is: {Poro:.2f}')

------------------------------------------------------------
    SOURCE     : openpnm.topotools._topotools.get_domain_area 
    TIME STAMP : 2022-07-13 16:47:17,025    
------------------------------------------------------------
------------------------------------------------------------
             boundary pores were not added 
    SOURCE     : openpnm.topotools._topotools.get_domain_length 
    TIME STAMP : 2022-07-13 16:47:17,038    
------------------------------------------------------------


The value of Porosity is: 0.21


### Discussions and Issues

One of the issues in estimation of porosity of the network is to estimate the domain volume correctly. In a cubic network for example, finding the length at x direction using Nx * spacing is erroneous. The reason is the actual domain length in x direction includes additional lengths from half of pore diameter for pores that locate on the left and right side. This issue becomes more problematic in extracted networks where the image data is not available and pore centers at the surface of the domain may not align at the same x coordinate. For such cases using OpenPNM's get_domain_area and get_domain_length is recommended to provide a better approximation of the domain volume.

Another issue is to ensure that the pore-scale models for volume are consistent. For example there is no overlapping pores, because in this case the void volume calculation will overestimate the real void space. Depending on the pore scale model used for the shape of pores and throats, they may need special methods to calculate the void volume to account for the overlap between throats and pores. For example in spheres and cylinders models, there is an overlap of pore volume in throats. Depending on the method that was used for assigning throat lengths, this overlap volume may be included in volume calculations. Existing methods to correct the throat volumes are ``lens`` and ``pendular_ring`` methods. See following example:

Pendular_ring method calculates the volume of the pendular rings residing between the end of a cylindrical throat and spherical pores that are in contact but not overlapping. This volume should be added to the throat volume if the throat length was
found as the center-to-center distance less the pore radii. Let's create a spheres and cylinders geometry:

In [5]:
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()

In [6]:
Vol_pendular_ring = np.sum(op.models.geometry.throat_volume.pendular_ring(pn))
Vol_void = np.sum(pn['pore.volume'])+np.sum(pn['throat.volume'])-Vol_pendular_ring
Poro = Vol_void / Vol_bulk
print(f'The value of Porosity is: {Poro:.2f}')

The value of Porosity is: 0.13


Although in this example the volume of pendular ring was negligible, depending on the size of the pores and throats, this value can be too high to be neglected.